Overview: This script creates visualizations for the gesture data. It outputs two type of html files: a figure for a single participant session or a matrix plot for a single gesture, which groups together several participant session figures onto one file. These files can be identified with the following html format:
    - single figure: (gesturetype)_(sessiontype)_Sub(subjectnumber)_Sess(sessionnumber)
        - Ex) Pan_Down_Freeform_Sub1_Sess1
    - all figures: (gesturetype)_(sessiontype)_Sub(firstsubjectnumber)_to_Sub(lastsubjectnumber)_Session_(totalnumberofparticipantsessions)_Grid_Page(pagenum)
        - Ex) Pan_Down_Freeform_Sub1_to_Sub26_Session_37_Grid_Page1

This tool utilizes the first code block for importing libraries and the two bottom code blocks to generate the visuals and requires the pandas and os libraries to be installed to run properly.

Read markdown for bottom two code blocks for more details and requirements.

In [2]:
# import necessary packages
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from plotnine import *
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler # Z-score variables
from sklearn.preprocessing import MinMaxScaler # Min-Max Normalization

from sklearn.model_selection import train_test_split # simple TT split cv

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import os

In [3]:
# Iterate through each CSV file in the specified folder
data_folder = "../AllData/F_RotForwardX"

# Create an empty list to store the dataframes
dfs = []

# Iterate over the files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(data_folder, filename))

        # Remove any data points that are not triggered by either the left or right trigger
        df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)
        
        # Extract the participant number from the file name (specified after "_subjID_")
        participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
        # Add the participant number as a column in the dataframe
        df.insert(0, 'participant_num', participant_num)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
all_F_rot_for_x_DF = pd.concat(dfs)

# Print the shape of the combined dataframe
print(all_F_rot_for_x_DF.shape)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../AllData/F_RotForwardX'

In [17]:
# Group the dataframe by participant number
grouped_by_participant = all_F_rot_for_x_DF.groupby('participant_num')

# Calculate the number of participants
num_participants = len(grouped_by_participant)

# Calculate the number of rows and columns based on the number of participants (sqrt number of participants and add 1)
rows = cols = int(num_participants ** 0.5) + 1 

# Create a subplot grid figure
fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'Participant {participant_num}' for participant_num, _ in grouped_by_participant], specs=[[{'type': 'scatter3d'}]*cols]*rows)

row = 1  # Starting row index for subplot
col = 1  # Starting column index for subplot

# Iterate over the grouped data by participant
for i, (participant_num, group) in enumerate(grouped_by_participant, start=1):
    # Iterate over trials and add scatter traces
    for trial_num, trial_group in group.groupby('gesture_counter'):
        fig.add_trace(go.Scatter3d(
            x=trial_group['r_controller_translation_x'],
            y=trial_group['r_controller_translation_z'],
            z=trial_group['r_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='viridis',  # colorscale
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Right Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
        fig.add_trace(go.Scatter3d(
            x=trial_group['l_controller_translation_x'],
            y=trial_group['l_controller_translation_z'],
            z=trial_group['l_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='sunset_r',  # colorscale (reverse)
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Left Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
    # Update subplot indices for the next participant
    col += 1
    if col > cols:
        col = 1
        row += 1

# Set overall title for the figure
fig.update_layout(height=1000, width=1000, title_text='All Participants Rotate Forward (Freeform)') 
# fig.show()

# Specify output file path
output_path = "../Figures/Rot_For_X_Sub_1_to_26_Grid.html"

# Save figure as an HTML file
pio.write_html(fig, file=output_path, auto_open=True)

In [19]:
# Iterate through each CSV file in the specified folder
data_folder = "../AllData/F_RotForwardX"

# Create an empty list to store the dataframes
dfs = []

# Create a dictionary to track the number of sessions for each participant
participant_sessions = {}

# Iterate over the files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(data_folder, filename))

        # Remove any data points that are not triggered by either the left or right trigger
        df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)

        # Extract the participant number from the file name (specified after "_subjID_")
        participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
        # Check if participant already has sessions
        if participant_num in participant_sessions:
            session_num = participant_sessions[participant_num] + 1
            participant_sessions[participant_num] = session_num
        else:
            session_num = 1
            participant_sessions[participant_num] = session_num
        
        # Add the participant number as a column in the dataframe
        df.insert(0, 'participant_num', participant_num)

        # Create a unique identifier for the session
        session_identifier = f"{session_num}"
        
        # Add the session identifier as a column in the dataframe
        df.insert(0, 'session_identifier', session_identifier)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
all_F_rot_for_x_DF = pd.concat(dfs)

# Print the shape of the combined dataframe
print(all_F_rot_for_x_DF.shape)

In [29]:
# Group the dataframe by participant number and session identifier
grouped_by_participant_session = all_F_rot_for_x_DF.groupby(['participant_num', 'session_identifier'])

# Calculate the number of participants and session
num_participants = len(grouped_by_participant_session)

# Calculate the number of rows and columns based on the number of participants and sessions
rows = cols = int(num_participants ** 0.5) + 1

# Create a subplot grid figure
fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'P {participant_num}, S {session_id}' for (participant_num, session_id), _ in grouped_by_participant_session], specs=[[{'type': 'scatter3d'}]*cols]*rows)

row = 1  # Starting row index for subplot
col = 1  # Starting column index for subplot

# Iterate over the grouped data by participant and session
for i, ((participant_num, session_id), group) in enumerate(grouped_by_participant_session, start=1):
    # Iterate over trials for each participant and session, and add scatter traces
    for trial_num, trial_group in group.groupby('gesture_counter'):
        # Add scatter trace for right controller
        fig.add_trace(go.Scatter3d(
            x=trial_group['r_controller_translation_x'],
            y=trial_group['r_controller_translation_z'],
            z=trial_group['r_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='viridis',  # colorscale
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Right Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
        # Add scatter trace for left controller
        fig.add_trace(go.Scatter3d(
            x=trial_group['l_controller_translation_x'],
            y=trial_group['l_controller_translation_z'],
            z=trial_group['l_controller_translation_y'],
            mode='markers',
            marker=dict(
                size=2,
                color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                colorscale='sunset_r',  # colorscale (reverse)
                opacity=0.8,
            ),
            name=f'Trial {trial_num} - Left Controller',
            showlegend=False  # hide legend entry
        ), row=row, col=col)
        
    # Update subplot indices for the next participant and session
    col += 1
    if col > cols:
        col = 1
        row += 1

# Set overall title for the figure
fig.update_layout(height=1000, width=1000, title_text='All Participants By Session Rotate Forward (Freeform)')  # Set the overall title for the figure
# fig.show()

# Specify the output file path
output_path = "C:\\Users\\vrelax\\Desktop\\VRelax\\gestureInterface\\Figures\\Rot_For_X_Sub_1_to_26_Session_37_Grid.html"

# Save the figure as an HTML file
pio.write_html(fig, file=output_path, auto_open=True)

ValueError: 
The 'specs' argument to make_subplots must be a 2D list of dictionaries with dimensions (1 x 1).
    Received value of type <class 'list'>: [[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]]

Overview: Creates subfolders in the Figures directory and converts the CleanedData directory into a dictionary for easier iteration.

Dictionary will have 13 keys to represent the 13 different gestures. Each key will have a value of a list that contains two lists. The two lists will represent the freeform and instruction folder respectively. Within each of those two lists, they will contain 37 data files paths. 
Ex: {'PanLeft': [../session_F_PanSelect_sub1ID.csv, ../session_F_PanSelect_sub2ID.csv, ..][../session_I_PanSelect_sub1ID.csv, ../session_I_PanSelect_sub2ID.csv, ..]}

Figures directory will have 13 empty gesture folders and 2 subfolders for freeform and instructional within each of them.

Requirements:
- The cleaned_data_folder_path and figures_folder_path variable must be edited to include the path to each of those folders.
- The Figures directory can be empty.
- Libraries in the first code block must be imported.
- The CleanedData directory must contain the following subfolders\files: ...\CleanedData\Sub#\Session_Sub#_Sess#\file.csv
    The names that are filled in are:
       - '#' in Sub# will be the subject's number
       - 'Session' in Session_Sub#_Sess# is either Freeform or Instructional, '#' in Sub# is the subject's number, and '#' in Sess# is the session number
       - 'file' in file.csv is the name of the file


In [4]:
'''
Edit paths here
'''
cleaned_data_folder_path = "C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData"
figures_folder_path = "C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\Figures\\FixedAxisModeWController"

'''
Iterates through CleanedData directory to make subfolders in Figures directory and make a dictionary of data where the 13 keys are gesture types 
and its values are two lists representing freeform and instructional. Each list will contain the path to the 37 csv data files.
'''
figure_gesture_dict = {}
for root, sub_folders, files in os.walk(cleaned_data_folder_path):
    for file in files:
        # Splits file name for gesture and session type identification
        file_split = file.split("_")
        file_gesture = file_split[3]
        file_session_type = file_split[2]
        
        # Ignores the thank you files
        if (file_session_type != 'F') and (file_session_type != 'I'):
            continue
        
        # Make gesture folder in Figures directory
        gesture_in_figures_path = os.path.join(figures_folder_path, file_gesture)
        if not os.path.exists(gesture_in_figures_path):
            os.makedirs(os.path.join(gesture_in_figures_path, "Freeform"))
            os.makedirs(os.path.join(gesture_in_figures_path, "Instructional"))

        # If a gesture is not in the dictionary, make the gesture a new key with list values freeform and instructional
        if file_gesture not in figure_gesture_dict:
            freeform_folder = []
            instructional_folder = []
            figure_gesture_dict[file_gesture] = [freeform_folder, instructional_folder]

        if file_session_type == 'F':
            figure_gesture_dict[file_gesture][0].append(os.path.join(root, file))
        else:
            figure_gesture_dict[file_gesture][1].append(os.path.join(root, file))

Overview: Creates graph visualizations for the gesture data files. Exports two types of html files: all_sessions and single_session. 

To clarify what is meant by a single_session, each single_session is grouped by participant session, so participants are paired by a unique session. A participant can have multiple sessions and each will be considered their own.
Examples: Box_Select_Sub1_Sess1, Box_Select_Sub2_Sess1, Box_Select_Sub2_Sess2

Files will be exported into their classified folders: Figures\GestureType\SessionType (GestureType & SessionType(Freeform/Instructional) will be edited accordingly)

single_session:
    - Contains a data visualization graph for a single grouped by participant session figure as well as 2 color bars representing both hands, which shows the gesture motion over time. 
    - The file can be identified as gesture_sessiontype_Sub#_Sess#.html with the 'gesture', 'sessiontype', and '#' being filled out. 
        - Example: 'Box_Select_Freeform_Sub1_Sess1.html'
all_sessions:
    - Each all_sessions file represents a single gesture and contain several individual single_session subplots for that specific gesture.
    - Each file has a max of 16 subplots to prevent overcrowded visuals, so an all_sessions file can be split up onto several different pages.
        - 37 visualizations will be separated into 16 subplots onto two pages and 5 subplots on the third page.
    - The file can be identified as gesture_sessiontype_Sub#_to_Sub#_Session_#.html with the 'gesture', 'sessiontype', and '#' being filled out.      
        - Example: 'Box_Select_Freeform_Sub1_to_Sub26_Session_37.html'

Requirements:
- The previous code block must successfully run to set up the variable figure_gesture_dict.

Fixed Axis Mode: All visualizations will have the same axis mode (data)

In [5]:
# Iterates through each gesture in the dictionary to create plots
for gesture in figure_gesture_dict:
    for session_type in figure_gesture_dict[gesture]:
        '''
        Reads each csv file into a dataframe and then concats all of the indivdual dataframes into one big one for a single gesture.
        '''
        # Create an empty list to store the dataframes
        dfs = []

        # Create a dictionary to track the number of sessions for each participant
        participant_sessions = {}

        for file_num in range(len(session_type)):
            # Read the CSV file into a dataframe
            df = pd.read_csv(session_type[file_num])
            filename = os.path.basename(session_type[file_num])

            # Remove any data points that are not triggered by either the left or right trigger
            df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)

            # Extract the participant number from the file name (specified after "_subjID_")
            participant_num = int(filename.split("_subjID_")[1].split("_")[0])
        
            # Check if participant already has sessions
            if participant_num in participant_sessions:
                session_num = participant_sessions[participant_num] + 1
                participant_sessions[participant_num] = session_num
            else:
                session_num = 1
                participant_sessions[participant_num] = session_num
        
            # Add the participant number as a column in the dataframe
            df.insert(0, 'participant_num', participant_num)

            # Create a unique identifier for the session
            session_identifier = f"{session_num}"
        
            # Add the session identifier as a column in the dataframe
            df.insert(0, 'session_identifier', session_identifier)
        
            # Append the dataframe to the list
            dfs.append(df)

        # Concatenate all the dataframes into a single dataframe
        all_data_for_gesture_DF = pd.concat(dfs)





        '''
        Groups dataframe by participant and session identifier and creates the figure's subplot template. 
        '''
        # Group the dataframe by participant number and session identifier
        grouped_by_participant_session = all_data_for_gesture_DF.groupby(['participant_num', 'session_identifier'])

        # Calculate the number of participants and session
        num_participants = len(grouped_by_participant_session)

        # Each file will have 4 rows and 4 columns. 
        # 37 visualizations will be split into three files with 16 of them in two files each and 5 in the third file.
        rows = cols = 4
        page_num = 1                    # The file num that the visualization will be on

        # Create a subplot grid figure
        fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'P {participant_num}, S {session_id}' for (participant_num, session_id), _ in grouped_by_participant_session], specs=[[{'type': 'scatter3d'}]*cols]*rows)

        # Starting row and column index for subplot
        row = 1  
        col = 1  

        '''
        Creates the title and output path for the all_sessions and single_session visualization files.
        '''
        gesture_name = ""
        session_name = ""
        html_file_name = ""
        single_session_output_path = figures_folder_path + "\\"
        all_sessions_output_path = figures_folder_path + "\\"
        all_sessions_title = "All Participants By Session "
        
        # Splits the first file in the folder by '_' to get the session and gesture type
        file = os.path.basename(session_type[0])
        session_split = file.split("_")[2]
        gesture_split = file.split("_")[3]

        # Adds to the output path string
        single_session_output_path += gesture_split + '\\'
        all_sessions_output_path += gesture_split + "\\"

        # Gets the session type and adds to the html output path
        if session_split == 'F':
            session_name = "Freeform"
            single_session_output_path += "Freeform\\"
            all_sessions_output_path += "Freeform\\"
        else:
            session_name = "Instructional"
            single_session_output_path += "Instructional\\"
            all_sessions_output_path += "Instructional\\"

        # Iterates through the gesture name. Separates the words with a space for the title and _ for the html output path
        for i in range(len(gesture)):
            if ((gesture[i].isupper() == True) and (i != 0)):
                gesture_name += " "
                html_file_name += "_"
            gesture_name += gesture[i]   
            html_file_name += gesture[i]

        '''
        Creates and exports visualization figures for all_sessions and single_session, represented by the variables fig and single_fig respectively. 

        The fig variable will contain all of the grouped by participant session figures. It can only contain up to 16 per file so if the file is full, fig
        will be exported to an html and resetted to an empty figure again to be filled by more subplots.

        The single_fig variable will be exported to an html after each single grouped by participant session figure has all of their trials traced. Each gesture
        will have 74 single_fig exports, 37 for freeform and another 37 for instructional.
        '''
        # Iterate over the grouped data by participant and session
        for i, ((participant_num, session_id), group) in enumerate(grouped_by_participant_session, start=1):
            # Single session figure different from all sessions figure, fig.
            single_fig = go.Figure()
            # Iterate over trials for each participant and session, and add scatter traces
            for trial_num, trial_group in group.groupby('gesture_counter'):
                # (All Sessions Figure) Add scatter trace for right controller
                fig.add_trace(go.Scatter3d(
                    x=trial_group['r_controller_translation_x'],
                    y=trial_group['r_controller_translation_y'],
                    z=trial_group['r_controller_translation_z'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='viridis',  # colorscale
                        opacity=0.8,
                    ),
                    name=f'Trial {trial_num} - Right Controller',
                    showlegend=False  # hide legend entry
                ), row=row, col=col)
                # (All Sessions Figure) Add scatter trace for left controller
                fig.add_trace(go.Scatter3d(
                    x=trial_group['l_controller_translation_x'],
                    y=trial_group['l_controller_translation_y'],
                    z=trial_group['l_controller_translation_z'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='sunset_r',  # colorscale (reverse)
                        opacity=0.8,
                    ),
                    name=f'Trial {trial_num} - Left Controller',
                    showlegend=False  # hide legend entry
                ), row=row, col=col)

                # (Single Session Figure) Add scatter trace and color bar for right controller
                single_fig.add_trace(go.Scatter3d(
                    x=trial_group['r_controller_translation_x'],
                    y=trial_group['r_controller_translation_y'],
                    z=trial_group['r_controller_translation_z'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='viridis',  # colorscale
                        opacity=0.8,
                        colorbar=dict(
                            title='Right',
                            tickvals=[trial_group.index[0], trial_group.index[-1]],
                            ticktext=['Start', 'End'],
                            len=0.6,
                            x=1.05,  # move the color bar to the left (adjust the value as needed)
                            y=.9,
                        )
                    ),
                    name=f'Trial {trial_num} - Right Controller',
                    showlegend=False
                ))
                # (Single Session Figure) Add scatter trace and color bar for left controller
                single_fig.add_trace(go.Scatter3d(
                    x=trial_group['l_controller_translation_x'],
                    y=trial_group['l_controller_translation_y'],
                    z=trial_group['l_controller_translation_z'],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=trial_group.index,  # set color based on occurrence (use index as a unique identifier)
                        colorscale='sunset_r',  # colorscale (reverse)
                        opacity=0.8,
                        colorbar=dict(
                            title='Left',
                            tickvals=[trial_group.index[0], trial_group.index[-1]],
                            ticktext=['Start', 'End'],
                            len=0.6,
                            x=.95,  # move the color bar to the left (adjust the value as needed)
                            y=0.9,
                        )
                    ),
                    name=f'Trial {trial_num} - Left Controller',
                    showlegend=False
                ))
                if np.any(trial_group['trigger_pull_amount_left'] > 0):
                    single_fig.update_traces(showlegend=True, selector = ({'name':f'Trial {trial_num} - Left Controller'}))
                if np.any(trial_group['trigger_pull_amount_right'] > 0):
                    single_fig.update_traces(showlegend=True, selector = ({'name':f'Trial {trial_num} - Right Controller'}))

                
                
            '''
            Export single_session figure
            '''
            # Update title for single_session figure
            single_session_title = gesture_name + ' (' + session_name + '): Participant ' + str(participant_num) + ' Session ' + session_id
            single_fig.update_layout(
                legend=dict(
                    x=.95,
                    y=-.5
                ),
                legend_title='Controllers in Use',
                title_text=single_session_title,
                scene_aspectmode='data'
            )
            
            # Create html for single session figure and exports single session figure
            temp_single_sess_output_path = single_session_output_path + html_file_name + '_' + session_name + '_Sub' + str(participant_num) + '_Sess' + session_id + '.html'
            if not os.path.exists(temp_single_sess_output_path):
                pio.write_html(single_fig, file=temp_single_sess_output_path, auto_open=False)


            # Update subplot indices for the next participant and session.
            col += 1
            if col > cols:
                col = 1
                row += 1

            '''
            Checks if the file is full for the all_session file (if row is greater than 4).
            If it is, current all_sessions file is exported, and the all_sessions figure will be resetted to take in more subplots for the next page
            of the file.
            '''
            if row == 5:
                # Update title and output path for all_sessions file
                temp_all_sessions_output_path = all_sessions_output_path + html_file_name + '_' + session_name + "_Sub1_to_Sub26_Session_37_Grid_Page" + str(page_num) + ".html"
                temp_all_sessions_title = all_sessions_title + gesture_name + " " + session_name + " (Page " + str(page_num) + ")"
                fig.update_layout(height=1000, width=1000, title_text=temp_all_sessions_title)    # Set the overall title for the figure
                fig.update_scenes(
                    aspectmode = 'data'
                )
                # Save the figure as an HTML file
                if not os.path.exists(temp_all_sessions_output_path):
                    pio.write_html(fig, file=temp_all_sessions_output_path, auto_open=False)

                # Create a new subplot grid figure.
                # Convert groupedby object into a dataframe to drop rows for grouped by participant session. This is done so that when the 3 different files 
                # are created for 37 visualizations, the subplot_titles don't restart back to subject 1 but rather where the previous file left off.
                end_index = page_num * 16
                df_grouped_by_participant_session = grouped_by_participant_session.aggregate(np.sum)
                df_grouped_by_participant_session = df_grouped_by_participant_session.drop(df_grouped_by_participant_session.index[0:end_index])
                grouped_by_participant_session2 = df_grouped_by_participant_session.groupby(['participant_num', 'session_identifier'])
                fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'P {participant_num}, S {session_id}' for (participant_num, session_id), _ in grouped_by_participant_session2], specs=[[{'type': 'scatter3d'}]*cols]*rows)

                # Restarts the new figure at row 1, col 1, and next page.
                row = 1
                col = 1
                page_num += 1

                
        # Updates and exports the final all_sessions file
        temp_all_sessions_output_path = all_sessions_output_path + html_file_name + '_' + session_name + "_Sub1_to_Sub26_Session_37_Grid_Page" + str(page_num) + ".html"
        temp_all_sessions_title = all_sessions_title + gesture_name + " " + session_name + " (Page " + str(page_num) + ")"
        fig.update_layout(height=1000, width=1000, title_text=temp_all_sessions_title)  # Set the overall title for the figure
        fig.update_scenes(
            aspectmode = 'data'
        )
        # Save the figure as an HTML file

        if not os.path.exists(temp_all_sessions_output_path):
            pio.write_html(fig, file=temp_all_sessions_output_path, auto_open=False)

BSPLINE TEST

In [33]:
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline


def drop_df(df, trial_num):
    df.drop(df[(df["trigger_pull_amount_right"] == 0) & (df["trigger_pull_amount_left"] == 0)].index, inplace=True)
    df.drop(df[(df['gesture_counter_UI']) != trial_num].index, inplace=True)
    df.reset_index(drop=True, inplace=True)


#path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv'
path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_PanDown_subjID_11_05-10-23_03-50-41.csv'
path_sess2 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_RotBackX_subjID_11_05-10-23_03-52-41.csv'
path_sess3 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv'
path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub1\\Freeform_Sub1_Sess1\\cleaned_session_F_PanDown_subjID_1_04-14-23_02-23-56.csv'
df_sess3 = pd.read_csv(path_sess3)
drop_df(df_sess3, 1)

# Gets new df
df_sess1 = pd.read_csv(path_sess1)
drop_df(df_sess1, 1)

fig1 = go.Figure()
sess1_rX = df_sess1['r_controller_translation_x']
sess1_rY = df_sess1['r_controller_translation_y']
sess1_rZ = df_sess1['r_controller_translation_z']

if np.any(df_sess1['trigger_pull_amount_left'] > 0):
    print('left hand')

x = np.array(sess1_rX)
y = np.array(sess1_rY)
z = np.array(sess1_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess1_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 5 * len(sess1_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 150 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])

fig1.add_trace(go.Scatter3d(
    x = sess1_rX,
    y = sess1_rY,
    z = sess1_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig1.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig1.update_layout(
    title_text='PAN DOWN SUB 11 SESS 1', 
    scene_aspectmode='data'
)
fig1.show()





# Gets new df
df_sess2 = pd.read_csv(path_sess2)
drop_df(df_sess2, 1)

fig2 = go.Figure()
sess2_rX = df_sess2['r_controller_translation_x']
sess2_rY = df_sess2['r_controller_translation_y']
sess2_rZ = df_sess2['r_controller_translation_z']

x = np.array(sess2_rX)
y = np.array(sess2_rY)
z = np.array(sess2_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess2_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 20 * len(sess2_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 150 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])
    

fig2.add_trace(go.Scatter3d(
    x = sess2_rX,
    y = sess2_rY,
    z = sess2_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig2.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig2.update_layout(
    title_text='ROT BACK X SUB 11 SESS 1', 
    scene_aspectmode='data'
)
fig2.show()

#fig2.update_layout(title_text='Smoothed: BOX SELECT SUB 1 SESS 1', scene_aspectmode='data')
#fig2.show()

In [100]:
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline


def drop_df(df, trial_num):
    df.drop(df[(df["trigger_pull_amount_right"] == 0) & (df["trigger_pull_amount_left"] == 0)].index, inplace=True)
    df.drop(df[(df['gesture_counter_UI']) != trial_num].index, inplace=True)
    df.reset_index(drop=True, inplace=True)


#path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv'

path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_PanRight_subjID_11_05-10-23_03-49-47.csv'
path_sess2 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_PanRight_subjID_11_06-05-23_03-21-29.csv'
path_sess3 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_PanRight_subjID_11_06-13-23_02-07-26.csv'
path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub10\\Freeform_Sub10_Sess1\\cleaned_session_F_PanRight_subjID_10_05-10-23_02-36-16.csv'
path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub4\\Freeform_Sub4_Sess1\\cleaned_session_F_PanRight_subjID_4_04-28-23_03-32-02.csv'
path_sess2 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub6\\Freeform_Sub6_Sess1\\cleaned_session_F_PanRight_subjID_6_05-05-23_03-31-23.csv'

# Gets new df
df_sess1 = pd.read_csv(path_sess1)
drop_df(df_sess1, 1)

fig1 = go.Figure()
sess1_rX = df_sess1['r_controller_translation_x']
sess1_rY = df_sess1['r_controller_translation_y']
sess1_rZ = df_sess1['r_controller_translation_z']

x = np.array(sess1_rX)
y = np.array(sess1_rY)
z = np.array(sess1_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess1_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 5 * len(sess1_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 50 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])

fig1.add_trace(go.Scatter3d(
    x = sess1_rX,
    y = sess1_rY,
    z = sess1_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig1.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig1.update_layout(
    title_text='PAN RIGHT SUB 4 SESS 1', 
    scene_aspectmode='data'
)
fig1.show()
print(len(sess1_rX))
print(len(x_smooth))




# Gets new df
df_sess2 = pd.read_csv(path_sess2)
drop_df(df_sess2, 4)

fig2 = go.Figure()
sess2_rX = df_sess2['r_controller_translation_x']
sess2_rY = df_sess2['r_controller_translation_y']
sess2_rZ = df_sess2['r_controller_translation_z']
print(len(sess2_rX))

x = np.array(sess2_rX)
y = np.array(sess2_rY)
z = np.array(sess2_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess2_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 5 * len(sess2_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 50 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])
    

fig2.add_trace(go.Scatter3d(
    x = sess2_rX,
    y = sess2_rY,
    z = sess2_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig2.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig2.update_layout(
    title_text='PAN RIGHT SUB 6 SESS 1', 
    scene_aspectmode='data'
)
fig2.show()
print(len(x_smooth))

#fig2.update_layout(title_text='Smoothed: BOX SELECT SUB 1 SESS 1', scene_aspectmode='data')
#fig2.show()

53
265
32


160


In [4]:
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline


def drop_df(df, trial_num):
    df.drop(df[(df["trigger_pull_amount_right"] == 0) & (df["trigger_pull_amount_left"] == 0)].index, inplace=True)
    df.drop(df[(df['gesture_counter_UI']) != trial_num].index, inplace=True)
    df.reset_index(drop=True, inplace=True)


#path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv'
path_sess1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_PanDown_subjID_11_05-10-23_03-50-41.csv'
path_sess2 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_RotBackX_subjID_11_05-10-23_03-52-41.csv'
path_sess3 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub11\\Freeform_Sub11_Sess1\\cleaned_session_F_BoxSelect_subjID_11_05-10-23_03-50-54.csv'
df_sess3 = pd.read_csv(path_sess3)
drop_df(df_sess3, 1)

# Gets new df
df_sess1 = pd.read_csv(path_sess1)
drop_df(df_sess1, 1)

fig1 = go.Figure()
sess1_rX = df_sess1['r_controller_translation_x']
sess1_rY = df_sess1['r_controller_translation_y']
sess1_rZ = df_sess1['r_controller_translation_z']

x = np.array(sess1_rX)
y = np.array(sess1_rY)
z = np.array(sess1_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess1_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 5 * len(sess1_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 150 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])

fig1.add_trace(go.Scatter3d(
    x = sess1_rX,
    y = sess1_rY,
    z = sess1_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig1.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig1.update_layout(
    title_text='PAN DOWN SUB 11 SESS 1', 
    scene_aspectmode='data'
)
fig1.show()





# Gets new df
df_sess2 = pd.read_csv(path_sess2)
drop_df(df_sess2, 1)

fig2 = go.Figure()
sess2_rX = df_sess2['r_controller_translation_x']
sess2_rY = df_sess2['r_controller_translation_y']
sess2_rZ = df_sess2['r_controller_translation_z']

x = np.array(sess2_rX)
y = np.array(sess2_rY)
z = np.array(sess2_rZ)

# The number of control points and knots
k = 3  # degree of the B-spline
t = np.linspace(0, 1, len(sess2_rZ))

# Create the B-spline representation for each dimension
spl_x = make_interp_spline(t, x, k=k)
spl_y = make_interp_spline(t, y, k=k)
spl_z = make_interp_spline(t, z, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t = np.linspace(0, 1, 20 * len(sess2_rZ))  # points in the smoothed curve; can be adjusted
x_smooth = spl_x(dense_t)
y_smooth = spl_y(dense_t)
z_smooth = spl_z(dense_t)

new_x_smooth = []
new_y_smooth = []
new_z_smooth = []
for index in range(len(x_smooth)):
    if (index % 150 != 0) :
        continue
    new_x_smooth.append(x_smooth[index])
    new_y_smooth.append(y_smooth[index])
    new_z_smooth.append(z_smooth[index])
    

fig2.add_trace(go.Scatter3d(
    x = sess2_rX,
    y = sess2_rY,
    z = sess2_rZ,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='ORIGINAL',
    showlegend=True)
)
fig2.add_trace(go.Scatter3d(
    x = new_x_smooth,
    y = new_y_smooth,
    z = new_z_smooth,
    mode='lines',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='SMOOTHED',
    showlegend=True)
)
fig2.update_layout(
    title_text='ROT BACK X SUB 11 SESS 1', 
    scene_aspectmode='data'
)
fig2.show()